# Relax Data Science Challenge

## Preparation

In [1]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Create dataframe of users
df_users = pd.read_csv('takehome_users.csv', encoding = "ISO-8859-1", parse_dates=['creation_time'])

In [3]:
# Create data fram of engagement
df_engagement = pd.read_csv('takehome_user_engagement.csv', encoding = "ISO-8859-1", parse_dates=['time_stamp'])

In [4]:
# Show first five users
df_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
# Show first five engagements
df_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


## New Columns

In [6]:
# Create date columns for users
df_users['day_of_week'] = df_users['creation_time'].dt.dayofweek
df_users['week_of_year'] = df_users['creation_time'].dt.week
df_users['year'] = df_users['creation_time'].dt.year
df_users['hour'] = df_users['creation_time'].dt.hour
df_users['month'] = df_users['creation_time'].dt.month
df_users['day_of_year'] = df_users['creation_time'].dt.dayofyear

In [7]:
# Create date columns for engagement
df_engagement['day_of_week'] = df_engagement['time_stamp'].dt.dayofweek
df_engagement['week_of_year'] = df_engagement['time_stamp'].dt.week
df_engagement['year'] = df_engagement['time_stamp'].dt.year
df_engagement['day_of_year'] = df_engagement['time_stamp'].dt.dayofyear

In [8]:
# Show df_engagement statistics
df_engagement.describe()

,user_id,visited,day_of_week,week_of_year,year,day_of_year
count,207917.000000,207917.0,207917.000000,207917.000000,207917.000000,207917.000000
mean,5913.314197,1.0,3.000313,23.908993,2013.377468,164.977332
std,3394.941674,0.0,2.001468,15.302984,0.590782,107.945743
min,1.000000,1.0,0.000000,1.000000,2012.000000,1.000000
25%,3087.000000,1.0,1.000000,11.000000,2013.000000,76.000000
50%,5682.000000,1.0,3.000000,20.000000,2013.000000,137.000000
75%,8944.000000,1.0,5.000000,38.000000,2014.000000,264.000000
max,12000.000000,1.0,6.000000,52.000000,2014.000000,366.000000


In [9]:
# Show df_engagement info
df_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 7 columns):
time_stamp      207917 non-null datetime64[ns]
user_id         207917 non-null int64
visited         207917 non-null int64
day_of_week     207917 non-null int64
week_of_year    207917 non-null int64
year            207917 non-null int64
day_of_year     207917 non-null int64
dtypes: datetime64[ns](1), int64(6)
memory usage: 11.1 MB


In [10]:
# Create total days column listing day the user is active
df_engagement['days'] = (df_engagement['year'] - 2012) * 365 + df_engagement['day_of_year'] 

In [11]:
# Repeat total days column for df_users listing day when account was created
df_users['days'] = (df_users['year'] - 2012) * 365 + df_users['day_of_year'] 

In [12]:
# Show first ten rows of df_engagement
df_engagement.head(10)

,time_stamp,user_id,visited,day_of_week,week_of_year,year,day_of_year,days
0,2014-04-22 03:53:30,1,1,1,17,2014,112,842
1,2013-11-15 03:45:04,2,1,4,46,2013,319,684
2,2013-11-29 03:45:04,2,1,4,48,2013,333,698
3,2013-12-09 03:45:04,2,1,0,50,2013,343,708
4,2013-12-25 03:45:04,2,1,2,52,2013,359,724
5,2013-12-31 03:45:04,2,1,1,1,2013,365,730
6,2014-01-08 03:45:04,2,1,2,2,2014,8,738
7,2014-02-03 03:45:04,2,1,0,6,2014,34,764
8,2014-02-08 03:45:04,2,1,5,6,2014,39,769
9,2014-02-09 03:45:04,2,1,6,6,2014,40,770


## Determine "Adopted Users"

In [13]:
# Create column that shows login two days later.
df_engagement['two_days_later'] = df_engagement.groupby('user_id').days.shift(2)

In [14]:
# Create column that shows difference between first and third logins
df_engagement['days_diff'] = df_engagement['days'] - df_engagement['two_days_later']

In [15]:
# Create dataframe of adopted users
df_engaged_users = df_engagement[df_engagement['days_diff'] <=7 ]

In [16]:
# Show dataframe of adopted users
df_engaged_users.head()

,time_stamp,user_id,visited,day_of_week,week_of_year,year,day_of_year,days,two_days_later,days_diff
9,2014-02-09 03:45:04,2,1,6,6,2014,40,770,764.0,6.0
10,2014-02-13 03:45:04,2,1,3,7,2014,44,774,769.0,5.0
11,2014-02-16 03:45:04,2,1,6,7,2014,47,777,770.0,7.0
24,2013-02-06 22:08:03,10,1,2,6,2013,37,402,395.0,7.0
27,2013-02-19 22:08:03,10,1,1,8,2013,50,415,410.0,5.0


In [17]:
# Create dataframe grouped by adopted user ids
df_engagement_3 = df_engaged_users.groupby(['user_id']).count()

In [18]:
# Show number of adopted users
len(df_engagement_3)

1656

In [19]:
# Show first five rows of grouped by dataframe
df_engagement_3.head()

,time_stamp,visited,day_of_week,week_of_year,year,day_of_year,days,two_days_later,days_diff
user_id,,,,,,,,,
2,3,3,3,3,3,3,3,3,3
10,267,267,267,267,267,267,267,267,267
20,1,1,1,1,1,1,1,1,1
33,4,4,4,4,4,4,4,4,4
42,329,329,329,329,329,329,329,329,329


In [20]:
# Get correct user ids
user_id = df_engagement_3.index.unique(level='user_id')

In [21]:
# Write function to determine adopted user
def adopted_user(row):
    if row['object_id'] in user_id:
        return 1
    else:
        return 0

In [22]:
# Apply adopted_user function to df_users dataframe
df_users['adopted_user'] = df_users.apply(adopted_user, axis=1)

In [23]:
# Show df_users with new adopted user column
df_users.head(10)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,day_of_week,week_of_year,year,hour,month,day_of_year,days,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1,17,2014,3,4,112,842,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,4,46,2013,3,11,319,684,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1,12,2013,23,3,78,443,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1,21,2013,8,5,141,506,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,3,3,2013,10,1,17,382,0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0,1,51,2013,3,12,351,716,0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN,6,50,2012,13,12,351,351,0
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN,2,31,2013,5,7,212,577,0
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN,1,45,2013,4,11,309,674,0
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,2,3,2013,22,1,16,381,1


## Correlations

In [24]:
corr_matrix = df_users.corr()
corr_matrix['adopted_user'].sort_values(ascending=False)

adopted_user                  1.000000
last_session_creation_time    0.392369
org_id                        0.066995
month                         0.038433
day_of_year                   0.037167
week_of_year                  0.036831
opted_in_to_mailing_list      0.008838
day_of_week                   0.006590
enabled_for_marketing_drip    0.006578
object_id                     0.005292
invited_by_user_id            0.001185
hour                          0.000086
year                         -0.085418
days                         -0.086246
Name: adopted_user, dtype: float64

The last session creation time is not relevant because adopted users are more likely to be active members. The year/days negative correlation at 8.5% indicates that users who signed up in the early part of this time period are more likely to be adopted users. This could be due to users who signed up later having fewer opportunities to become adopted users.

It's also fairly clear that it made no difference for users who opted into mailing list, were enabled for marketing drip, and were invited by a user_id.

The scatter plot above is not very convincing, but it does appear that the most recent users are more likely not to have adopted yet. This could be due to fewer opportunities.

In [25]:
# Show distribution of creation_source for all users
df_users['creation_source'].value_counts(normalize=True)

ORG_INVITE            0.354500
GUEST_INVITE          0.180250
PERSONAL_PROJECTS     0.175917
SIGNUP                0.173917
SIGNUP_GOOGLE_AUTH    0.115417
Name: creation_source, dtype: float64

In [26]:
# Create dataframe of adopted users only
df_adopted_users = df_users[df_users['adopted_user']==1]

In [27]:
# Show distribution of creation source for adopted users only
df_adopted_users['creation_source'].value_counts(normalize=True)

ORG_INVITE            0.346618
GUEST_INVITE          0.222826
SIGNUP                0.182367
SIGNUP_GOOGLE_AUTH    0.144324
PERSONAL_PROJECTS     0.103865
Name: creation_source, dtype: float64

Guest invites are more likey to become adopted users. Personal projects are less likely to become adopted users. Signup with Google Authorization is a little more likely to become an adopted user.